In [1]:
import os
import shutil
import random

# Paths (adjust these to match your Kaggle dataset paths)
image_dir = "/kaggle/input/fawryvideos/train/images"
label_dir = "/kaggle/input/fawryvideos/train/labels"
dataset_dir = "/kaggle/working"

# Create train/val directories for images and labels
for split in ["train", "val"]:
    os.makedirs(os.path.join(dataset_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(dataset_dir, "labels", split), exist_ok=True)

# List all image files and sort them
all_images = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
print(f"Total images found: {len(all_images)}")
# # Filter: only select every 50th frame (e.g. frame 000050, 000100, etc.)
# selected_images = []
# for img in all_images:
#     try:
#         # Extract the numeric part from the filename (e.g., "000050" from "000050.jpg")
#         # 1 frame from each 25 for video 02
#         # 1 frame from each 50 for videos 03 & 05
#         frame_number = int(os.path.splitext(img)[0])
#         if frame_number % 25 == 0:
#             selected_images.append(img)
#     except ValueError:
#         # If the filename doesn't represent a number, skip it.
#         continue

# Shuffle the selected images
random.shuffle(all_images)
# print(f"Images after filtering every 25th frame: {len(selected_images)}")

# Split: 80% train, 20% val
train_size = int(0.8 * len(all_images))
train_images = all_images[:train_size]
val_images = all_images[train_size:]

def copy_files(image_list, split):
    for img in image_list:
        # Copy the image
        src_img = os.path.join(image_dir, img)
        dst_img = os.path.join(dataset_dir, "images", split, img)
        shutil.copy(src_img, dst_img)
        
        # Derive the corresponding label file name
        label_filename = os.path.splitext(img)[0] + ".txt"
        src_label = os.path.join(label_dir, label_filename)
        dst_label = os.path.join(dataset_dir, "labels", split, label_filename)
        
        # Only copy the label if it exists (ensuring we filter labels as well)
        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)
        else:
            print(f"Label file {src_label} not found. Skipping label for {img}.")

# Copy files for train and validation splits
copy_files(train_images, "train")
copy_files(val_images, "val")
print(f"Images after filtering every 25th frame: {len(train_images)}")
print(f"Images after filtering every 25th frame: {len(val_images)}")

print("Dataset split complete.")

Total images found: 389
Images after filtering every 25th frame: 311
Images after filtering every 25th frame: 78
Dataset split complete.


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 15.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 7.7 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

In [ ]:
import os
import torch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()  # clear any unused memory

from ultralytics import YOLO

# Enable mixed precision training if supported (check documentation if additional flag is needed)
model = YOLO("/kaggle/input/yolov11x/pytorch/default/1/yolo11x.pt")
data_yaml = "/kaggle/input/yaml-without-mot/data.yaml"

# Train with a lower batch size, lower image size, and consider using gradient accumulation if available.
model.train(
    data=data_yaml,
    epochs=128,
    imgsz=640,      # lower image resolution
    batch=16,        # further reduce batch size
    name='custom_yolov11_retail',
    patience=5
)

custom_save_path = "/kaggle/working/custom_yolov11_retail_best_imgsz.pt"
model.save(custom_save_path)
print(f"Model weights saved to {custom_save_path}")

# Optionally, if your training loop supports gradient accumulation, check the documentation on how to enable it.


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/yolov11x/pytorch/default/1/yolo11x.pt, data=/kaggle/input/yaml-without-mot/data.yaml, epochs=128, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=custom_yolov11_retail, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, spli

100%|██████████| 755k/755k [00:00<00:00, 16.6MB/s]
E0000 00:00:1745024325.435467      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745024325.558425      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 68.0MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1607.1±465.0 MB/s, size: 67.3 KB)


...

      6/128      14.5G       1.66      1.461      1.683        240        640:  85%|████████▌ | 17/20 [00:26<00:04,  1.51s/it]

      6/128      14.5G      1.652       1.46      1.683         97        640: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.26it/s]

                   all         78        912          0          0          0          0
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



6 epochs completed in 0.052 hours.
Optimizer stripped from runs/detect/custom_yolov11_retail/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/custom_yolov11_retail/weights/best.pt, 114.4MB

Validating runs/detect/custom_yolov11_retail/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x summary (fused): 190 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 1/3 [00:01<00:02,  1.33s/it]

### Inferenece

In [7]:
import os
import json
from ultralytics import YOLO

# Step 1: Load the trained YOLO model
model_path = "/kaggle/working/custom_yolov11_retail_best_imgsz.pt"
model = YOLO(model_path)

# Step 2: Define the data YAML file (it should contain paths to your test/val images and labels)
data_yaml = "/kaggle/input/yamlfile2/data.yaml"

# Step 3: Evaluate the model
# The .val() method runs inference on the test/validation split and computes common metrics (e.g., mAP)
results = model.val(data=data_yaml)

# The results object typically contains a metrics dictionary. Print evaluation metrics.
print("Evaluation Metrics:")
print(results.metrics)

# Step 4: Save the evaluation metrics to a JSON file for further analysis
eval_results_path = "/kaggle/working/evaluation_results.json"
with open(eval_results_path, "w") as f:
    json.dump(results.metrics, f, indent=4)

print(f"Evaluation results saved to {eval_results_path}")


Ultralytics 8.3.95 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /kaggle/input/selected-images/img_02/dataset/labels/train... 181 images, 0 backgrounds, 0 corrupt: 100%|██████████| 181/181 [00:00<00:00, 531.51it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/selected-images/img_02/dataset/labels is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:10<00:00,  1.20it/s]


                   all        181      20561      0.922      0.858      0.931      0.616


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 1.3ms preprocess, 3.7ms inference, 0.0ms loss, 12.8ms postprocess per image
Results saved to runs/detect/val5
Evaluation Metrics:


AttributeError: 'DetMetrics' object has no attribute 'metrics'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class.
        names (dict): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (dict): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
    